## Inception otherwise known as GoogLeNet Straight out of the paper

In [1]:
%autosave 300
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch


In [3]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchview import draw_graph
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
from torcheval.metrics import MulticlassAccuracy, BinaryAccuracy
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import (
    OneCycleLR,
    StepLR,
    ExponentialLR,
)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import scikitplot as skplt
import seaborn as sns
import torch.nn.functional as F
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import cv2
from torch_lr_finder import LRFinder

In [5]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [6]:
def plot_loss_accuracy(
    train_loss,
    val_loss,
    train_acc,
    val_acc,
    labels,
    colors,
    loss_legend_loc="upper center",
    acc_legend_loc="upper left",
    legend_font=5,
    fig_size=(16, 10),
    sub_plot1=(1, 2, 1),
    sub_plot2=(1, 2, 2),
):

    plt.rcParams["figure.figsize"] = fig_size
    plt.figure

    plt.subplot(sub_plot1[0], sub_plot1[1], sub_plot1[2])

    for i in range(len(train_loss)):
        x_train = range(len(train_loss[i]))
        x_val = range(len(val_loss[i]))

        min_train_loss = np.array(train_loss[i]).min()

        min_val_loss = np.array(val_loss[i]).min()

        plt.plot(
            x_train,
            train_loss[i],
            linestyle="-",
            color="tab:{}".format(colors[i]),
            label="TRAIN ({0:.4}): {1}".format(min_train_loss, labels[i]),
        )
        plt.plot(
            x_val,
            val_loss[i],
            linestyle="--",
            color="tab:{}".format(colors[i]),
            label="VALID ({0:.4}): {1}".format(min_val_loss, labels[i]),
        )

    plt.xlabel("epoch no.")
    plt.ylabel("loss")
    plt.legend(loc=loss_legend_loc, prop={"size": legend_font})
    plt.title("Training and Validation Loss")

    plt.subplot(sub_plot2[0], sub_plot2[1], sub_plot2[2])

    for i in range(len(train_acc)):
        x_train = range(len(train_acc[i]))
        x_val = range(len(val_acc[i]))

        max_train_acc = np.array(train_acc[i]).max()

        max_val_acc = np.array(val_acc[i]).max()

        plt.plot(
            x_train,
            train_acc[i],
            linestyle="-",
            color="tab:{}".format(colors[i]),
            label="TRAIN ({0:.4}): {1}".format(max_train_acc, labels[i]),
        )
        plt.plot(
            x_val,
            val_acc[i],
            linestyle="--",
            color="tab:{}".format(colors[i]),
            label="VALID ({0:.4}): {1}".format(max_val_acc, labels[i]),
        )

    plt.xlabel("epoch no.")
    plt.ylabel("accuracy")
    plt.legend(loc=acc_legend_loc, prop={"size": legend_font})
    plt.title("Training and Validation Accuracy")

    plt.show()
    plt.close()

In [7]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(
        self,
        patience=3,
        verbose=True,
        delta=1e-5,
        trace_func=print,
        path="models",
        model_name="model.pt",
    ):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.model_name = model_name
        self.trace_func = trace_func

    def __call__(self, val_loss, model, epoch):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(
                f"EarlyStopping counter: {self.counter} out of {self.patience}"
            )
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            self.trace_func(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        os.makedirs(self.path, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(self.path, self.model_name))
        self.val_loss_min = val_loss


def get_device():
    """Get device (if GPU is available, use GPU)"""
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = get_device()
print(device)
batch_size = 512
epochs = 25
learning_rate = 1e-3

Random seed set as 42
cuda


In [9]:
# Specify the image classes
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [21]:
# transform for the dataset using Compose from albumentations
def data_augmentations():
    """Data Augmentations for the CIFAR10 dataset using albumentations library"""
    means = [0.4914, 0.4822, 0.4465]
    stds = [0.2470, 0.2435, 0.2616]

    train_transforms = alb.Compose(
        [
            alb.Normalize(mean=means, std=stds, always_apply=True),
            alb.PadIfNeeded(min_height=40, min_width=40, always_apply=True),
            alb.RandomCrop(height=32, width=32, always_apply=True),
            alb.HorizontalFlip(),
            alb.CoarseDropout(
                max_holes=1,
                max_height=8,
                max_width=8,
                min_holes=1,
                min_height=8,
                min_width=8,
                fill_value=means,
            ),
            ToTensorV2(),
        ]
    )

    test_transforms = alb.Compose(
        [
            alb.Normalize(mean=means, std=stds, always_apply=True),
            ToTensorV2(),
        ]
    )
    return train_transforms, test_transforms

In [22]:
# downloading train and test data
train = datasets.CIFAR10("./data", train=True, download=True)
test = datasets.CIFAR10("./data", train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
class CifarDS(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label

In [24]:
train_transforms, test_transforms = data_augmentations()
train_dateset = CifarDS(train.data, train.targets, train_transforms)
test_dateset = CifarDS(test.data, test.targets, test_transforms)

In [25]:
def get_data_loader(
    train_dateset, test_dateset, train_transforms, test_transforms, dataloader_args
):
    """Data loader for the CIFAR10 dataset"""
    # Loading custom datasets
    train_dateset = CifarDS(train_dateset.data, train_dateset.targets, train_transforms)
    test_dateset = CifarDS(test_dateset.data, test_dateset.targets, test_transforms)
    # train dataloader
    train_loader = torch.utils.data.DataLoader(train_dateset, **dataloader_args)
    # test dataloader
    test_loader = torch.utils.data.DataLoader(test_dateset, **dataloader_args)
    return train_loader, test_loader

In [26]:
for data, label in train_dateset:
    print(data.shape)
    print(label)
    print(data.mean())
    break

torch.Size([3, 32, 32])
6
tensor(-0.3312)


In [27]:
# Dataloaders
# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = (
    dict(shuffle=True, batch_size=batch_size, num_workers=4, pin_memory=True)
    if device.type == "cuda"
    else dict(shuffle=True, batch_size=batch_size)
)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_dateset, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_dateset, **dataloader_args)

In [28]:
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break

/anaconda/envs/torch_env/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/anaconda/envs/torch_env/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([512, 3, 32, 32])
torch.Size([512])


## Inception Lenet Model

https://miro.medium.com/v2/resize:fit:828/format:webp/1*yGcmBL1Nyqpj2lNG2sj5Wg.png

Conv Block for the dataprep part 

In [ ]:
class ConvBlock(nn.Module):
    """Convolutional Block for Initial data prep for Inception block"""

    def __int__(self, in_channels, out_channels, **kwargs):
        super(ConvBlock, self).__int__()
        self.convb = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.convb(x)
        x = self.bn(x)
        x = F.relu(x)
        return x

Next we design the inception block which is the most important part we throw the model three options: one-by-one, three-by-three, and five-by-five kernels, and we let the model figure out how to weigh and process information from these kernels.

In [29]:
class InceptionBlock(nn.Module):
    """Inception Block for the model"""

    def __init__(self, in_channels, out_11, in_33, out_33, in_55, out_55, out_pool):
        super(InceptionBlock, self).__init__()
        # 1*1 block
        self.branch11 = nn.Conv2d(in_channels, out_11, kernel_size=1)
        # 1*1 -> 3*3 block
        self.branch33 = nn.Sequential(
            nn.Conv2d(in_channels, in_33, kernel_size=1, padding=0),
            nn.Conv2d(in_33, out_33, kernel_size=3, padding=1),
        )
        # 1*1 -> 5*5 block
        self.branch55 = nn.Sequential(
            nn.Conv2d(in_channels, in_55, kernel_size=1, padding=0),
            nn.Conv2d(in_55, out_55, kernel_size=5, padding=2),
        )
        # maxpool -> 1*1 block
        self.branchpool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, out_pool, kernel_size=1),
        )

    def forward(self, x):
        return torch.cat(
            [self.branch11(x), self.branch33(x), self.branch55(x), self.branchpool(x)],
            1,
        )

In [33]:
inception_block = InceptionBlock(192, 64, 96, 128, 16, 32, 32).to(device)
summary(inception_block, input_size=(1,192, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
InceptionBlock                           [1, 256, 32, 32]          --
├─Conv2d: 1-1                            [1, 64, 32, 32]           12,352
├─Sequential: 1-2                        [1, 128, 32, 32]          --
│    └─Conv2d: 2-1                       [1, 96, 32, 32]           18,528
│    └─Conv2d: 2-2                       [1, 128, 32, 32]          110,720
├─Sequential: 1-3                        [1, 32, 32, 32]           --
│    └─Conv2d: 2-3                       [1, 16, 32, 32]           3,088
│    └─Conv2d: 2-4                       [1, 32, 32, 32]           12,832
├─Sequential: 1-4                        [1, 32, 32, 32]           --
│    └─MaxPool2d: 2-5                    [1, 192, 32, 32]          --
│    └─Conv2d: 2-6                       [1, 32, 32, 32]           6,176
Total params: 163,696
Trainable params: 163,696
Non-trainable params: 0
Total mult-adds (M): 167.62
Input size (MB): 0.79
Forward/ba

Next we will define the auxillary classifiers which will be optional in the main Model